<a href="https://colab.research.google.com/github/DayalStrub/ecir2021tutorial/blob/main/other/2-simplest-thing-WIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install -q python-terrier
!pip install -q git+https://github.com/terrier-org/pyterrier.git

     |████████████████████████████████| 1.1MB 9.4MB/s 
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 163kB 20.1MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 5.5MB 26.6MB/s 
     |████████████████████████████████| 133kB 47.3MB/s 
     |████████████████████████████████| 1.8MB 46.3MB/s 
     |████████████████████████████████| 645kB 36.3MB/s 
     |████████████████████████████████| 81kB 8.1MB/s 


In [2]:
import pyterrier as pt
pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


terrier-assemblies 5.4  jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.5  jar not found, downloading to /root/.pyterrier...
Done
terrier-prf -SNAPSHOT  jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [3]:
dataset = pt.get_dataset("vaswani")
index_ref = dataset.get_index()
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

In [8]:
queries = dataset.get_topics()
queries.head()

,qid,query
0,1,measurement of dielectric constant of liquids ...
1,2,mathematical analysis and design details of wa...
2,3,use of digital computers in the design of band...
3,4,systems of data coding for information transfer
4,5,use of programs in engineering testing of comp...


In [7]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
rm3 = pt.rewrite.RM3(index)

rm3_pipe = bm25 >> rm3 >> bm25

In [ ]:
# TODO Question: in rm3_pipe, is final bm25 just reranking initial bm25 results, or does it re-consider all docs?

In [ ]:
# TODO figure out how to implement PR Feedback using logistic regression on bag of words a la Lin

In [17]:
# NOTE indeed/actually a function
pt.Experiment(
    retr_systems=[bm25, rm3_pipe],
    names=["BM25", "BM25-RM3-BM25",],
    topices=queries,
    qrels=dataset.get_qrels(),
    eval_metrics=["map"]
              )

,name,map
0,BR(BM25),0.296517
1,"Compose(Compose(BR(BM25), <pyterrier.rewrite.R...",0.293326
